# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from scipy.stats import randint
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [2]:
#Memuat dataset hasil clustering
df = pd.read_csv('Clustered_Customers.csv')
print(df.head())

   CustomerID  Gender  Age  Annual Income ($)  Spending Score (1-100)  \
0           1    Male   19            15000.0                      39   
1           2    Male   21            35000.0                      81   
2           3  Female   20            86000.0                       6   
3           4  Female   23            59000.0                      77   
4           5  Female   31            38000.0                      40   

      Profession  Work Experience  Family Size  Cluster  
0     Healthcare                1            4        2  
1       Engineer                3            3        2  
2       Engineer                1            1        1  
3         Lawyer                0            2        2  
4  Entertainment                2            6        2  


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [3]:
# Pisahkan fitur (X) dan target (y)
X = df.drop(columns=['Cluster', 'CustomerID'])
y = df['Cluster']

In [4]:
# Encoding data categorical
categorical_cols = ['Gender', 'Profession']
numerical_cols = X.columns.difference(categorical_cols)

# Membuat pipeline untuk jika terdapat menangani missing values, encoding, dan scaling
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numerical_cols),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder())
        ]), categorical_cols)
    ])

# Menggunakan pipeline preprocessor
X_processed = preprocessor.fit_transform(X)

# Membagi data menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.3, random_state=42)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [5]:
#Mengatasi ketidakseimbangan data dengan SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Mengurangi dimensi dengan PCA (jaga 95% variansi)
pca = PCA(n_components=0.95)
X_train_resampled = pca.fit_transform(X_train_resampled)
X_test = pca.transform(X_test)

# Parameter distribusi untuk Random Forest
rf_param_dist = {
    'n_estimators': randint(50, 150),
    'max_depth': randint(5, 20),
    'min_samples_split': randint(4, 10),
    'min_samples_leaf': randint(2, 5),
    'bootstrap': [True, False]
}

# RandomizedSearchCV untuk Random Forest dengan cv=10
rf_random_search = RandomizedSearchCV(RandomForestClassifier(random_state=42), rf_param_dist, n_iter=10, cv=10,
                                      scoring='f1_weighted', n_jobs=-1, random_state=42)
rf_random_search.fit(X_train_resampled, y_train_resampled)

# Menampilkan hyperparameter terbaik untuk Random Forest
print("Best parameters for Random Forest:", rf_random_search.best_params_)

# Model terbaik dari RandomizedSearchCV
rf_best_model = rf_random_search.best_estimator_

# Evaluasi model pada data testing
rf_y_pred = rf_best_model.predict(X_test)

Best parameters for Random Forest: {'bootstrap': False, 'max_depth': 19, 'min_samples_leaf': 3, 'min_samples_split': 9, 'n_estimators': 96}


Tulis narasi atau penjelasan algoritma yang Anda gunakan.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [6]:
# Evaluasi hasil model Random Forest
print("\nRandom Forest Classification Report:")
print(classification_report(y_test, rf_y_pred))

rf_f1 = f1_score(y_test, rf_y_pred, average='weighted')
rf_accuracy = accuracy_score(y_test, rf_y_pred)
print(f"Random Forest F1-score: {rf_f1:.2f}")
print(f"Random Forest Accuracy: {rf_accuracy:.2f}")


Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       212
           1       0.92      0.85      0.88       217
           2       0.86      0.93      0.90       161

    accuracy                           0.92       590
   macro avg       0.91      0.92      0.92       590
weighted avg       0.92      0.92      0.92       590

Random Forest F1-score: 0.92
Random Forest Accuracy: 0.92


In [8]:
# Melakukan cross-validation
cv_scores = cross_val_score(rf_best_model, X_train_resampled, y_train_resampled, cv=5, scoring='f1_weighted')

# Menampilkan skor rata-rata cross-validation
print("Cross-Validation Scores:", cv_scores)
print("Mean Cross-Validation Score:", cv_scores.mean())

Cross-Validation Scores: [0.89564305 0.91932953 0.88848044 0.90628635 0.89362792]
Mean Cross-Validation Score: 0.9006734580551395


Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

# 1. Precision atau Recall Rendah untuk Kelas Tertentu

Berdasarkan **classification report** yang saya dapatkan, terlihat bahwa **precision** dan **recall** untuk masing-masing kelas memiliki perbedaan yang signifikan:

- **Kelas 0** (Precision: 0.96, Recall: 0.98): Hasilnya cukup memuaskan karena memiliki precision dan recall yang sangat baik.
- **Kelas 1** (Precision: 0.92, Recall: 0.85): Recall di sini agak rendah dibandingkan precision, yang berarti model agak kesulitan untuk mengenali kelas ini dengan baik.
- **Kelas 2** (Precision: 0.86, Recall: 0.93): Meskipun recall cukup tinggi, precision agak rendah, yang menunjukkan model cukup baik dalam mendeteksi kelas ini tetapi sering memberikan prediksi positif yang salah.

Saya merasa bahwa masalah utama ada pada kelas 1, di mana recall lebih rendah daripada precision, yang mungkin disebabkan oleh ketidakseimbangan data yang belum sepenuhnya terselesaikan meskipun saya sudah menggunakan **SMOTE** untuk menangani masalah ini.



# 2. Apakah Model Mengalami Overfitting atau Underfitting?

Dari hasil **cross-validation scores** yang saya peroleh, dengan rata-rata sekitar 0.90, model saya tampaknya cukup stabil. F1-score antara data training dan testing tidak terlalu berbeda jauh, yang menunjukkan bahwa model ini bisa menggeneralisasi dengan baik. Ini berarti model tidak menunjukkan tanda-tanda **overfitting** atau **underfitting** yang jelas.

Namun, saya tetap perlu waspada terhadap **overfitting** jika ada kesenjangan besar antara performa di data training dan testing, atau jika model terlalu kompleks karena banyaknya hyperparameter yang dioptimalkan. Untuk lebih meyakinkan, saya bisa mencoba melihat **learning curve** atau menggunakan teknik regulasi lain seperti **dropout** atau **penalti**.



# 3. Rekomendasi Tindakan Lanjutan

- **Mengatasi Ketidakseimbangan Data**: Walaupun saya sudah menggunakan SMOTE, saya merasa ketidakseimbangan data masih menjadi masalah. Beberapa langkah yang bisa saya coba adalah:
  - Meningkatkan **tuning SMOTE** untuk mendapatkan sampel yang lebih seimbang dan mengurangi potensi overfitting pada kelas minoritas.
  - Menerapkan **penalti pada kelas minoritas** (misalnya, dengan menggunakan parameter `class_weight` di Random Forest) untuk memberikan bobot lebih pada kelas yang kurang terwakili.

- **Pengumpulan Data Tambahan**: Jika kelas tertentu, seperti kelas 1, masih menunjukkan performa yang buruk, saya pikir pengumpulan data lebih banyak untuk kelas tersebut dapat sangat membantu untuk meningkatkan akurasi model.

- **Cobalah Algoritma Lain**: Walaupun Random Forest cukup baik, saya berpikir ada kemungkinan algoritma lain bisa memberikan hasil yang lebih baik:
  - **XGBoost** atau **LightGBM**, yang lebih kuat dalam menangani ketidakseimbangan data dan sering memberikan hasil yang lebih baik pada dataset dengan noise.
  - **SVM** dengan **kernel non-linear** juga bisa menjadi pilihan jika model lain lebih cocok untuk dataset ini.

- **Pengoptimalan Hyperparameter**: Saya sudah mencoba **RandomizedSearchCV**, tetapi saya bisa memperbanyak iterasi atau mencoba teknik lain seperti **Bayesian optimization** untuk menemukan parameter yang lebih optimal.

